In [ ]:
using Pkg
Pkg.activate("..")
import CCSPOMDPs

pomdp = CCSPOMDPs.CCSPOMDP();


In [ ]:
visualize_uncertainty(pomdp, 1, :topSealThickness)

In [ ]:
visualize_gt(pomdp, 2)

### Test: Action function returns all actions

In [ ]:
all_actions = POMDPs.actions(pomdp);

### Test: Observation function returns correct distributions

In [ ]:
POMDPs.observation(pomdp, all_actions[2], pomdp.state)

In [ ]:
POMDPs.reward(pomdp, pomdp.state, all_actions[2])

In [ ]:
visualize_uncertainty(pomdp, 2, :topSealThickness)

In [ ]:
POMDPs.observation(pomdp, all_actions[21], pomdp.state) # Can take up to 7 mins to run!

In [ ]:
POMDPs.reward(pomdp, pomdp.state, all_actions[21])

In [ ]:
visualize_uncertainty(pomdp, 2, :z)

### Test: Points close are predicted with more accuracy and less uncertainty than points far.

In [ ]:
gtlayer = pomdp.state.earth[1].gt

# seismic_line = (x1=0.0, y1=0.0, x2=5.0, y2=98.0)
well1 = Point(19, 34)
well2 = Point(30, 12)
well_close = Point(31, 14)
well_far = Point(98, 97)

all_wells = Multi([well1, well2])
data_at_all_wells = gtlayer[all_wells, :]

γ = SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET)

okrig = GeoStatsModels.OrdinaryKriging(γ)
fitkrig = GeoStatsModels.fit(okrig, data_at_all_wells)

# We notice that prediction at close has much less variance than prediction at far, 
# and as an additional sanity check the prediction at close is close to the well2 point.
# Krigging predictprob only does one attribute at a time.

probs_far = GeoStatsModels.predictprob(fitkrig, :z, well_far)
probs_close = GeoStatsModels.predictprob(fitkrig, :z, well_close)
probs_close, probs_far

### Test: Uncertainty is low around known points.
Additionally, as nugget is increased uncertainty (both globally and at known points) increases.
Pay attention to colorbar when verifying this.

### Buy well data at 3-4 locations and visualize belief

In [ ]:
buy_well_data(pomdp, 2)
buy_well_data(pomdp, 5)
buy_well_data(pomdp, 7)

In [ ]:
visualize_uncertainty(Multi([pomdp.collected_locs...]), 3, "z") # Note this is time consuming at just 3 gathered points.

In [ ]:
distr = observe(pomdp, Point(18, 24), 3, "z")
rand(distr)

In [ ]:
# A scratch cell to run directly.
using Reexport
using Distributions
using Statistics
using GeoStats
using Random
using DataFrames
import GLMakie as Mke
using Infiltrator
using AbstractGPs
using Unitful
using Plots
using DataStructures
include("../src/utils.jl")
include("../src/config.jl");